# Use Existing Statistical Profiler in CLI
"profile_statistics" is available as an analysis pass in Machop CLI. Here we show how to use it to profile the statistics the weight and activations of a resnet18 pretrained on ImageNet.

## Example: the range of weights & input activations of a `Conv2d` node

Say we want to collect the tensor-wise min-max range of the 1st `torch.nn.Conv2d` nodes' weights & bias, and the channel-wise 97% quantile min-max of the 1st `torch.nn.Conv2d` nodes' input activations. We can do the following:

> **Note**
> - Tensor-wise min-max of weights: `Conv2d` has a weight tensor of shape `(out_channels, in_channels, kernel_size, kernel_size)`, so we need to reduce all dimensions of the weight tensor.
> - Channel-wise 97% quantile min-max of activations: `Conv2d` has an activation tensor of shape `(batch_size, in_channels, height, width)`, so we need to reduce the `0`-th, `2`-nd, and `3`-rd dimensions of the activation tensor. 97% quantile min-max means we sort the input activation tensors ascendingly, and take the min at 3% of the sorted tensor, and the max at 97% of the sorted tensor.

### Create a config toml for Machop CLI

In [18]:
import toml

config = {
    "model": "resnet18",
    "dataset": "cifar10",  # we use cifar10 as an example since imagenet is too large
    "batch_size": 4,
    "pretrained": True,
    "passes": {
        "profile_statistics": {
            "by": "name",  # collect statistics by node name
            "target_weight_nodes": [
                "conv1"
            ],  # the 1st conv2d node name is "conv1". collect the statistics of the weight tensor
            "target_activation_nodes": [
                "conv1"
            ],  # collects the statistics of the 1st conv2d node
            "weight_statistics": {
                # collect the min-max range of the weight tensor
                "range_min_max": {
                    "dims": "all",  # reduce all dimensions
                    "abs": False,  # do not take the absolute value before min max reduction
                }
            },
            "activation_statistics": {
                "range_quantile": {
                    "dims": [
                        0,
                        2,
                        3,
                    ],  # reduced dim = 0, 2, 3. The min-max is a 1D tensor of shape (C_in,)
                    "abs": False,
                    "quantile": 0.97,  # take the 97% quantile
                }
            },
            "num_samples": 32,  # feed 32 samples to the model
        },
        "report_node_meta_param": {"which": ["software"]},
        "save_node_meta_param": {"save_path": "node_software_meta_param_cli.toml"},
    },
}

# Save the config to a toml file
with open("config.toml", "w") as f:
    toml.dump(config, f)

Here is how the CLI config toml looks like:

the entry `passes.profile_statistics` specifies the profiler behavior. 

In [1]:
! cat config.toml

model = "resnet18"
dataset = "cifar10"
batch_size = 4
pretrained = true

[passes.profile_statistics]
by = "name"
target_weight_nodes = [ "conv1",]
target_activation_nodes = [ "conv1",]
num_samples = 32

[passes.report_node_meta_param]
which = [ "software",]

[passes.save_node_meta_param]
save_path = "node_software_meta_param_cli.toml"

[passes.profile_statistics.weight_statistics.range_min_max]
dims = "all"
abs = false

[passes.profile_statistics.activation_statistics.range_quantile]
dims = [ 0, 2, 3,]
abs = false
quantile = 0.97


### Run CLI

In [22]:
! ../../../../../machop/ch transform --config ./config.toml

/home/zz7522/anaconda3/envs/mase-sw/lib/python3.10/site-packages/torch/cuda/__init__.py:546: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
Global seed set to 0
+-------------------------+--------------------------+--------------+-----------------+--------------------------+
| Name                    |         Default          | Config. File | Manual Override |        Effective         |
+-------------------------+--------------------------+--------------+-----------------+--------------------------+
| task                    |      classification      |              |                 |      classification      |
| load_name               |           None           |              |                 |           None           |
| load_type               |            mz            |              |                 |            mz            |
| batch_size              |           128            |      4       |                 |            4             |
| to_

## A More Complex Example

Here is an example of collecting various statistics of a ResNet18 model by node type.

In [2]:
import toml

config = {
    "model": "resnet18",
    "dataset": "cifar10",  # we use cifar10 as an example since imagenet is too large
    "batch_size": 4,
    "pretrained": True,
    "passes": {
        "profile_statistics": {
            "by": "type",
            "target_weight_nodes": [
                "linear",
                "conv2d",
                "batch_norm2d",
                "adaptive_avg_pool2d",
                "relu",
            ],
            "target_activation_nodes": [
                "linear",
                "conv2d",
                "batch_norm2d",
                "adaptive_avg_pool2d",
                "relu",
            ],
            "weight_statistics": {
                # "record": {"device": "cuda"},
                "variance_online": {"device": "cuda", "dims": "all"},
                "variance_precise": {"device": "cuda", "dims": "all"},
                "range_n_sigma": {
                    "device": "cuda",
                    "dims": "all",
                    "abs": False,
                    "var_mode": "precise",
                    "num_sigma": 3,
                },
                "range_min_max": {"device": "cuda", "dims": "all", "abs": False},
                "range_quantile": {
                    "device": "cuda",
                    "dims": "all",
                    "abs": False,
                    "quantile": 0.97,
                },
            },
            "activation_statistics": {
                # "record": {"device": "cuda"},
                "variance_online": {"device": "cuda", "dims": "all"},
                "variance_precise": {"device": "cuda", "dims": "all"},
                "range_n_sigma": {
                    "device": "cuda",
                    "dims": "all",
                    "abs": False,
                    "var_mode": "precise",
                    "num_sigma": 3,
                },
                "range_min_max": {"device": "cuda", "dims": "all", "abs": False},
                "range_quantile": {
                    "device": "cuda",
                    "dims": "all",
                    "abs": False,
                    "quantile": 0.97,
                },
            },
            "num_samples": 32,
        },
        "report_node_meta_param": {"which": ["software"]},
        "save_node_meta_param": {"save_path": "node_software_meta_param_cli_complex.toml"},
    },
}

# Save the config to a toml file
with open("config_complex.toml", "w") as f:
    toml.dump(config, f)

In [1]:
! ../../../../../machop/ch transform --config ./config_complex.toml

/home/zz7522/anaconda3/envs/mase-sw/lib/python3.10/site-packages/torch/cuda/__init__.py:546: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
Global seed set to 0
+-------------------------+--------------------------+--------------+-----------------+--------------------------+
| Name                    |         Default          | Config. File | Manual Override |        Effective         |
+-------------------------+--------------------------+--------------+-----------------+--------------------------+
| task                    |      classification      |              |                 |      classification      |
| load_name               |           None           |              |                 |           None           |
| load_type               |            mz            |              |                 |            mz            |
| batch_size              |           128            |      4       |                 |            4             |
| to_